In [1]:
import music21
import pickle
import numpy as np
from music21 import duration

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


### Load data from files

https://verovio.humdrum.org/?file=essen/europa/deutschl/altdeu2/deut4207.krn

In [3]:
data = []

In [4]:
files = ["data_germany_1","data_germany_2","data_germany_3","data_germany_4","data_germany_5","data_germany_6","data_germany_7", "data_germany_8", "data_germany_9", "data_germany_10", "data_germany_11"]
for file in files:
    with open('../data/data_2/{}.pkl'.format(file), 'rb') as handle:
        data.extend(pickle.load(handle))

### Extracting the data

In [5]:
notes_input = []
durations_input = []
phrases_input = []

for piece in data:
    notes = piece[0]
    notes_input.append(129) # START = 129
    notes_input.extend(notes)

    durations = piece[1]
    durations_input.append(0)
    durations_input.extend(durations)

    phrases = piece[2]
    phrases_input.append(0)
    phrases_input.extend(phrases)

In [6]:
print("Length of the notes_input: {}".format(len(notes_input)))
print("Twenty first elements: {}".format(notes_input[:20]))
print("Length of the durations_input: {}".format(len(durations_input)))
print("Twenty first elements: {}".format(durations_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_input)))
print("Twenty first elements: {}".format(phrases_input[:20]))

Length of the notes_input: 282505
Twenty first elements: [129, 55, 60, 60, 60, 60, 60, 62, 64, 64, 64, 67, 62, 128, 67, 60, 128, 55, 60, 60]
Length of the durations_input: 282505
Twenty first elements: [0, 0.5, 0.5, 0.5, 0.375, 0.125, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5]
Length of the phrases_input: 282505
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [7]:
durations_to_int = {}
int_to_duration = {}
notes_to_int = {}
int_to_notes = {}
phrases_to_int = {}
int_to_phrases = {}
for index, duration in enumerate(sorted(set(durations_input))):
    durations_to_int[duration] = index
    int_to_duration[index] = duration

for index, note in enumerate(sorted(set(notes_input))):
    notes_to_int[note] = index
    int_to_notes[index] = note

for index, phrase in enumerate(sorted(set(phrases_input))):
    phrases_to_int[phrase] = index
    int_to_phrases[index] = phrase

durations_network_input = [ durations_to_int[duration] for duration in durations_input ]
notes_network_input = [ notes_to_int[note] for note in notes_input ]
phrases_network_input = [ phrases_to_int[phrase] for phrase in phrases_input ]

In [8]:
int_to_duration

{0: 0,
 1: 0.00390625,
 2: 0.125,
 3: Fraction(1, 6),
 4: 0.25,
 5: Fraction(1, 3),
 6: 0.375,
 7: 0.5,
 8: Fraction(2, 3),
 9: 0.75,
 10: 1.0,
 11: Fraction(4, 3),
 12: 1.5,
 13: 2.0,
 14: 3.0,
 15: 4.0,
 16: 6.0,
 17: 8.0,
 18: 12.0}

In [9]:
print("Length of the notes_input: {}".format(len(notes_network_input)))
print("Twenty first elements: {}".format(notes_network_input[:20]))
print("Length of the durations_input: {}".format(len(durations_network_input)))
print("Twenty first elements: {}".format(durations_network_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_network_input)))
print("Twenty first elements: {}".format(phrases_network_input[:20]))

Length of the notes_input: 282505
Twenty first elements: [47, 16, 21, 21, 21, 21, 21, 23, 25, 25, 25, 28, 23, 46, 28, 21, 46, 16, 21, 21]
Length of the durations_input: 282505
Twenty first elements: [0, 7, 7, 7, 6, 2, 4, 4, 7, 7, 7, 7, 10, 7, 7, 10, 7, 7, 7, 7]
Length of the phrases_input: 282505
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [10]:
n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)

In [11]:
print("Number of distinct notes: {}".format(n_notes))
print("Number of distinct durations: {}".format(n_durations))

Number of distinct notes: 48
Number of distinct durations: 19


In [12]:
with open('dictionary.pkl', 'wb') as file:
    dictionary = { "durations_to_int" : durations_to_int, "int_to_duration" : int_to_duration, "notes_to_int" : notes_to_int, "int_to_notes" : int_to_notes, "phrases_to_int" : phrases_to_int, "int_to_phrases" : int_to_phrases }
    # A new file will be created
    pickle.dump(dictionary, file)

In [13]:
import os
import pickle
import numpy
from music21 import note, chord

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
from RNNmodel import prepare_sequences, create_network

In [14]:
notes_network_input

[47,
 16,
 21,
 21,
 21,
 21,
 21,
 23,
 25,
 25,
 25,
 28,
 23,
 46,
 28,
 21,
 46,
 16,
 21,
 21,
 21,
 21,
 23,
 25,
 25,
 25,
 28,
 23,
 46,
 28,
 21,
 46,
 28,
 33,
 32,
 30,
 28,
 26,
 25,
 46,
 25,
 25,
 46,
 30,
 28,
 26,
 46,
 25,
 26,
 46,
 30,
 28,
 25,
 46,
 24,
 25,
 46,
 28,
 33,
 32,
 30,
 28,
 26,
 25,
 46,
 24,
 25,
 46,
 28,
 28,
 30,
 30,
 30,
 32,
 30,
 32,
 33,
 33,
 46,
 47,
 19,
 24,
 24,
 24,
 23,
 26,
 24,
 23,
 21,
 21,
 29,
 23,
 46,
 19,
 24,
 46,
 19,
 24,
 24,
 24,
 23,
 26,
 24,
 23,
 21,
 21,
 29,
 23,
 46,
 19,
 24,
 46,
 19,
 24,
 24,
 24,
 24,
 23,
 24,
 26,
 21,
 21,
 21,
 46,
 21,
 23,
 23,
 23,
 23,
 21,
 23,
 24,
 26,
 28,
 46,
 19,
 24,
 24,
 24,
 24,
 23,
 24,
 26,
 21,
 21,
 21,
 46,
 21,
 23,
 23,
 23,
 23,
 21,
 23,
 24,
 24,
 46,
 47,
 19,
 24,
 24,
 24,
 24,
 26,
 24,
 23,
 21,
 21,
 29,
 23,
 46,
 28,
 24,
 46,
 19,
 19,
 24,
 24,
 24,
 24,
 26,
 24,
 23,
 21,
 21,
 29,
 23,
 46,
 28,
 24,
 46,
 47,
 16,
 21,
 21,
 21,
 23,
 25,
 26,
 30,


In [15]:
durations_network_input

[0,
 7,
 7,
 7,
 6,
 2,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 7,
 7,
 7,
 7,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 4,
 4,
 13,
 10,
 13,
 10,
 10,
 7,
 7,
 10,
 10,
 10,
 10,
 10,
 7,
 7,
 10,
 10,
 10,
 10,
 10,
 7,
 7,
 10,
 10,
 10,
 10,
 13,
 10,
 13,
 10,
 10,
 7,
 7,
 10,
 1,
 7,
 7,
 12,
 7,
 10,
 10,
 10,
 10,
 14,
 10,
 10,
 0,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 7,
 7,
 7,
 6,
 2,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 7,
 12,
 7,
 10,
 10,
 10,
 10,
 12,
 7,
 10,
 10,
 10,
 10,
 12,
 7,
 10,
 10,
 10,
 10,
 13,
 10,
 10,
 10,
 10,
 12,
 7,
 10,
 10,
 10,
 10,
 12,
 7,
 10,
 10,
 10,
 10,
 12,
 7,
 10,
 10,
 10,
 10,
 14,
 10,
 12,
 0,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 4,
 4,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 0,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 0,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 4,
 4,
 6,
 

In [16]:
phrases_network_input

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


Remove phrases that are too short or too long.

In [17]:
len(notes_network_input)

282505

In [18]:
len(durations_network_input)

282505

In [19]:
len(phrases_network_input)

282505

In [20]:
phrases_network_input

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


In [23]:
# keep phrases with lengths 4 - 12
def select_phrases(phrases_network_input, notes_network_input, durations_network_input, lower_bound=4, upper_bound=12):
    new_phrases_network_input = []
    new_notes_network_input = []
    new_durations_network_input = []

    current_length = 0

    tmp_phrase = []
    tmp_note = []
    tmp_duration = []

    for i, digit in enumerate(phrases_network_input):
        current_length += 1
        tmp_phrase.append(digit)
        tmp_note.append(notes_network_input[i])
        tmp_duration.append(durations_network_input[i])
        if digit == 1:
            if current_length >= lower_bound and current_length <= upper_bound:
                new_phrases_network_input.extend(tmp_phrase)
                new_notes_network_input.extend(tmp_note)
                new_durations_network_input.extend(tmp_duration)
            current_length = 0
            tmp_phrase.clear()
            tmp_note.clear()
            tmp_duration.clear()

    return new_phrases_network_input, new_notes_network_input, new_durations_network_input


In [24]:
new_phrases_network_input, new_notes_network_input, new_durations_network_input = select_phrases(phrases_network_input, notes_network_input, durations_network_input, lower_bound=4, upper_bound=12)

In [25]:
len(new_notes_network_input)

27850

In [26]:
new_notes_network_input

[47,
 16,
 21,
 21,
 21,
 21,
 21,
 23,
 25,
 25,
 25,
 28,
 23,
 46,
 28,
 21,
 46,
 16,
 21,
 21,
 21,
 21,
 23,
 25,
 25,
 25,
 28,
 23,
 46,
 28,
 21,
 46,
 46,
 30,
 28,
 26,
 46,
 25,
 26,
 46,
 30,
 28,
 25,
 46,
 24,
 25,
 46,
 28,
 33,
 32,
 30,
 28,
 26,
 25,
 46,
 24,
 25,
 46,
 28,
 28,
 30,
 30,
 30,
 32,
 30,
 32,
 33,
 33,
 46,
 47,
 19,
 24,
 24,
 24,
 23,
 26,
 24,
 23,
 21,
 21,
 29,
 23,
 46,
 19,
 24,
 46,
 19,
 24,
 24,
 24,
 23,
 26,
 24,
 23,
 21,
 21,
 29,
 23,
 46,
 19,
 24,
 46,
 21,
 46,
 46,
 21,
 46,
 47,
 19,
 24,
 24,
 24,
 24,
 26,
 24,
 23,
 21,
 21,
 29,
 23,
 46,
 28,
 24,
 46,
 19,
 19,
 24,
 24,
 24,
 24,
 26,
 24,
 23,
 21,
 21,
 29,
 23,
 46,
 28,
 24,
 46,
 47,
 16,
 21,
 21,
 21,
 23,
 25,
 26,
 30,
 28,
 25,
 28,
 23,
 46,
 25,
 21,
 46,
 47,
 19,
 24,
 24,
 28,
 26,
 24,
 23,
 21,
 21,
 21,
 29,
 23,
 46,
 31,
 24,
 46,
 19,
 19,
 24,
 24,
 24,
 24,
 28,
 26,
 24,
 23,
 21,
 21,
 21,
 29,
 23,
 46,
 31,
 24,
 46,
 21,
 26,
 11,
 16,
 16,
 16,


In [27]:
len(new_durations_network_input)

27850

In [28]:
new_durations_network_input

[0,
 7,
 7,
 7,
 6,
 2,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 7,
 7,
 7,
 7,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 10,
 10,
 10,
 10,
 7,
 7,
 10,
 10,
 10,
 10,
 10,
 7,
 7,
 10,
 10,
 10,
 10,
 13,
 10,
 13,
 10,
 10,
 7,
 7,
 10,
 1,
 7,
 7,
 12,
 7,
 10,
 10,
 10,
 10,
 14,
 10,
 10,
 0,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 7,
 7,
 7,
 6,
 2,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 10,
 10,
 10,
 10,
 10,
 0,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 4,
 4,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 0,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 0,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 4,
 4,
 6,
 2,
 4,
 4,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 10,
 13,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 4,
 4,
 7,
 7,
 4,
 4,
 4,
 4,
 7,
 7,
 7,
 7,
 10,
 7,
 7,
 10,
 7,
 4,
 4,
 7,
 4,
 4,
 7,
 4,


In [29]:
len(new_phrases_network_input)

27850

In [30]:
new_phrases_network_input

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


### End ops

In [31]:
seq_len = 5
network_input, network_output = prepare_sequences(new_notes_network_input, new_durations_network_input, new_phrases_network_input, seq_len)

In [32]:
print('note input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('phrases_output')
print(network_output[0][0])

note input
[47 16 21 21 21]
duration input
[0 7 7 7 6]
phrases_output
[1. 0.]


In [33]:
embed_size = 100
rnn_units = 256
use_attention = True
model, att_model = create_network(n_notes, n_durations, n_phrases, embed_size, rnn_units, use_attention)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    4800        ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    1900        ['input_2[0][0]']                
                                                                                              

/Users/almazur/miniconda3/envs/nn/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [34]:
import numpy as np
weights_folder = "weights" #os.path.join(run_folder, 'weights')
# model.load_weights(os.path.join(weights_folder, "weights.h5"))


checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=20, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )

Epoch 1/20


2023-05-09 11:22:17.741521: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


697/697 [==============================] - 16s 20ms/step - loss: 0.5428 - val_loss: 0.5168
Epoch 2/20
697/697 [==============================] - 16s 23ms/step - loss: 0.5147 - val_loss: 0.5187
Epoch 3/20
697/697 [==============================] - 17s 24ms/step - loss: 0.5034 - val_loss: 0.4817
Epoch 4/20
697/697 [==============================] - 18s 26ms/step - loss: 0.4958 - val_loss: 0.4909
Epoch 5/20
697/697 [==============================] - 17s 24ms/step - loss: 0.4891 - val_loss: 0.4730
Epoch 6/20
697/697 [==============================] - 16s 23ms/step - loss: 0.4817 - val_loss: 0.4793
Epoch 7/20
697/697 [==============================] - 15s 22ms/step - loss: 0.4754 - val_loss: 0.4843
Epoch 8/20
697/697 [==============================] - 17s 24ms/step - loss: 0.4659 - val_loss: 0.4915
Epoch 9/20
697/697 [==============================] - 17s 25ms/step - loss: 0.4534 - val_loss: 0.4902
Epoch 10/20
697/697 [==============================] - 17s 24ms/step - loss: 0.4385 - val_los